https://qiita.com/cvusk/items/285e2b02b0950537b65e
    

In [2]:
import numpy as np

from sklearn import datasets, preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.utils import np_utils
from keras import backend as K
from keras.wrappers.scikit_learn import KerasClassifier

/home/yuki/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
iris = datasets.load_iris()

X = preprocessing.scale(iris.data)
y = np_utils.to_categorical(iris.target)
print(X.shape, y.shape)

(150, 4) (150, 3)


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   train_size=0.7)
num_classes = y_test.shape[1]

/home/yuki/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [6]:
num_classes

3

In [7]:
print(X_train.shape, X_test.shape)

(105, 4) (45, 4)


In [9]:
def iris_model(activation='relu', optimizer="adam",
              out_dim=100):
    model = Sequential()
    model.add(Dense(out_dim, input_dim=4, activation=activation))
    model.add(Dense(out_dim, activation=activation))
    model.add(Dense(num_classes, activation="softmax"))
    model.compile(loss='categorical_crossentropy',
                 optimizer=optimizer, 
                 metrics=['accuracy'])
    return model
    

In [14]:
activation = ['relu', 'sigmoid']
optimizer = ['adam', 'adagrad']
out_dim = [100, 200]
nb_epoch = [10, 25]
batch_size = [5, 10]

In [17]:
model = KerasClassifier(build_fn=iris_model, verbose=0)
param_grid = dict(activation=activation,
                 optimizer=optimizer,
                 out_dim=out_dim,
                 nb_epoch=nb_epoch,
                 batch_size=batch_size)
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(X_train, y_train)

In [18]:
grid_result.best_score_

0.8666666746139526

In [19]:
grid_result.best_params_

{'activation': 'relu',
 'batch_size': 5,
 'nb_epoch': 10,
 'optimizer': 'adagrad',
 'out_dim': 200}

In [20]:
model = iris_model(activation=grid_result.best_params_['activation'], 
                   optimizer=grid_result.best_params_['optimizer'], 
                   out_dim=grid_result.best_params_['out_dim'])
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_412 (Dense)            (None, 200)               1000      
_________________________________________________________________
dense_413 (Dense)            (None, 200)               40200     
_________________________________________________________________
dense_414 (Dense)            (None, 3)                 603       
Total params: 41,803
Trainable params: 41,803
Non-trainable params: 0
_________________________________________________________________
